In [ ]:
import pandas as pd
import numpy as np
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import time

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE


url = "https://ieeexplore.ieee.org/search/searchresult.jsp?action=search&newsearch=true&matchBoolean=true&queryText=(%22Author%20Affiliations%22:{})&ranges=1960_2023_Year)"

def html_call(url):
    html = urllib.request.urlopen(url, context=ctx).read()
    soup = BeautifulSoup(html, 'html.parser')
    print(soup.find_all('h1 _ngcontent-vem-c102="" class="Dashboard-header col-12"'))


<H4>GET THE DATA INTO PYTHON TO BE MANIPULATED AND CREATE THE URL LIST</H4>

In [ ]:
m = pd.read_excel('')                            #INSERT THE NAME OF THE EXCEL FILE WITH THE LIST OF DEVICE MAKERS.
m.head()
                                                 # convert the items in the dataframe to a list of strings
list_of_strings = []
for index, row in m.iterrows():
    list_of_strings.append(' '.join([str(row[col]) for col in m.columns]))
print(list_of_strings)
urls = list()
for company in list_of_strings:
    urls.append(url.format(company))             # CREATES A LIST OF URLS FOR THE WEBSCRAPER TO ITERATE THROUGH
print(len(urls))

<H4>CODE TO CHECK IF THE URL WORKS</H4>

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
driver = webdriver.Chrome()
import time
i = 1
for company in urls:
    driver.get(company)
    time.sleep(3)              #DELAY ADDED IN TO MAKE SURE THE USER CAN SEE THE PAGE LONG ENOUGH TO NOTICE IT




<H4>FINALLY, USE SELENIUM TO INTERACT WITH THE WEBSITE</H4>

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set up the web driver
driver = webdriver.Chrome()

for company in urls:
    driver.get(company)
    # Wait for the Export button to be clickable and click it
    try:
         export_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div/div/div[3]/div/xpl-root/main/div/xpl-search-results/div/div[1]/div[1]/ul/li[1]/xpl-export-search-results/button"))
)
         export_button.click()
    except:
         try:
             export_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[5]/div/div/div[3]/div/xpl-root/main/div/xpl-search-results/div/div[1]/div[1]/ul/li[2]/xpl-export-search-results/button")))
             export_button.click()
         except:
            print('Company not found: ' + company)
            continue


         

    
# Get the current `body` tag element
    body = driver.find_element(By.TAG_NAME, "body")
# Use JavaScript to modify the `style` attribute of the `body` tag
    driver.execute_script("arguments[0].setAttribute('style', 'overflow: visible;')", body)

# Wait for the modal title to appear
    wait = WebDriverWait(driver, 20)
    title = wait.until(EC.visibility_of_element_located((By.XPATH, "/html/body/ngb-modal-window/div/div")))
    export_csv = driver.find_element(By.XPATH, "/html/body/ngb-modal-window/div/div/div[2]/div/div/div[2]/button")
    export_csv.click()
    WebDriverWait(driver, 200).until(EC.invisibility_of_element_located((By.XPATH, "/html/body/ngb-modal-window/div/div/div[2]/div/div/div[2]/button")))



<h4>ADD THE DIRECTORY OF THE DOWNLOADED FILES INTO A PATH VARIABLE AND ADD IN A COMPANY COLUMN TO EVERY FILE</h4>

In [ ]:
import glob
import pandas as pd
from pathlib import Path
q = list()
i = 0
path = r"C:\Users\Ayushman\Downloads\test_2\*.csv"
for fname in glob.glob(path):
    x = pd.read_csv(fname, sep='\t')
    x['Company'] = list_of_strings[i]
    i += 1
    x.to_csv(fname)

<H4>COMBINE ALL THE CSV FILES INTO ONE EXCEL FILE</H4>